# Carga com API do Spotify para a camada Bronze(Dados brutos)

## Importacao de LIBs
- requests → faz requisições HTTP (GET, POST etc.) para acessar APIs ou sites.
- base64 → codifica e decodifica dados em formato Base64 (ex: para autenticação em APIs).
- concurrent.futures (ThreadPoolExecutor, as_completed) → executa tarefas em paralelo (threads) para acelerar loops, como várias chamadas de API.
- json → lê e grava dados no formato JSON (muito usado em APIs).
- pandas → manipula e analisa dados em tabelas (DataFrames).
- time → mede ou controla o tempo (ex: sleep, medir duração de execução).

In [0]:
import requests
import base64
from concurrent.futures import ThreadPoolExecutor, as_completed
import json
import pandas as pd
import time

## Verificando se a delta table existe
## Limpando delta table da search_track_artist da camada Bronze

In [0]:
if spark.catalog.tableExists("workspace.bronze.search_track_artist"):
    spark.sql("TRUNCATE TABLE workspace.bronze.search_track_artist")
else:
    print("Tabela não existe. Nenhuma ação foi tomada.")


## Autenticação

In [0]:
# Função para pegar token
def get_spotify_access_token(client_id, client_secret):
    url = "https://accounts.spotify.com/api/token"
    headers = {
        "Authorization": "Basic " + base64.b64encode(
            f"{client_id}:{client_secret}".encode()
        ).decode()
    }
    data = {"grant_type": "client_credentials"}
    response = requests.post(url, headers=headers, data=data)
    response.raise_for_status()
    token_info = response.json()
    return token_info["access_token"], time.time() + token_info["expires_in"] - 60  # expira em timestamp


## Renovação do token da Autenticação

In [0]:
# Função para renovar token se necessário
def get_valid_token(client_id, client_secret, token, token_expiry):
    """
    Garante que sempre exista um token válido do Spotify API.

    Args:
        client_id (str): Client ID da aplicação registrada no Spotify.
        client_secret (str): Client Secret da aplicação registrada no Spotify.
        token (str): Token de acesso atual (pode estar expirado).
        token_expiry (float): Timestamp (em segundos desde epoch) indicando quando o token expira.

    Returns:
        tuple:
            - token (str): Token válido (renovado se necessário).
            - token_expiry (float): Novo timestamp de expiração do token.
    
    Funcionamento:
        - Verifica se o horário atual (`time.time()`) já passou do `token_expiry`.
        - Se o token já expirou:
            → Imprime uma mensagem de log ("Token expirou. Renovando...").
            → Chama a função `get_spotify_access_token()` para obter um novo token e atualizar o `token_expiry`.
        - Se o token ainda for válido:
            → Apenas retorna o token atual e seu tempo de expiração.
    """
    if time.time() >= token_expiry:
        print("Token expirou. Renovando...")
        token, token_expiry = get_spotify_access_token(client_id, client_secret)
    return token, token_expiry


## Buscando tracks(musicas) utilizando API do Spotify

In [0]:
# Função para buscar música (tratando rate limit e token expirado)
def get_spotify_track_info(track_name, client_id, client_secret, token, token_expiry):
    """
    Busca informações de uma música específica na API do Spotify,
    garantindo o uso de um token válido e tratando limites de taxa (rate limit).

    Args:
        track_name (str): Nome da música a ser pesquisada.
        client_id (str): Client ID da aplicação Spotify.
        client_secret (str): Client Secret da aplicação Spotify.
        token (str): Token de acesso atual (pode estar expirado ou inválido).
        token_expiry (float): Timestamp indicando quando o token expira.

    Returns:
        tuple:
            - list: Lista de resultados (máximo 1 item) retornada pela API do Spotify.
            - str: Token de acesso válido (renovado se necessário).
            - float: Novo timestamp de expiração do token.

    Funcionamento:
        1. Garante que o token está válido chamando `get_valid_token()`.
        2. Faz a chamada para o endpoint `/v1/search` do Spotify:
            - Query: `track:{track_name}`
            - Tipo: `track`
            - Limit: 1 (apenas o resultado mais relevante).
        3. Analisa a resposta:
            - **200 (OK):** Retorna a lista de tracks (máximo 1).
            - **429 (Rate Limit):**
                → API limitou requisições.  
                → Lê o header `Retry-After` para saber quantos segundos esperar.  
                → Faz `time.sleep()` e tenta de novo.
            - **401 (Unauthorized):**
                → Token inválido ou expirado.  
                → Pede um novo token com `get_spotify_access_token()` e repete a busca.
            - **Outros códigos de erro:**
                → Imprime mensagem de erro (`Erro {status_code}`) e retorna lista vazia.
    """
    while True:
        token, token_expiry = get_valid_token(client_id, client_secret, token, token_expiry)
        url = "https://api.spotify.com/v1/search"
        headers = {"Authorization": f"Bearer {token}"}
        params = {"q": f"track:{track_name}", "type": "track", "limit": 1}

        response = requests.get(url, headers=headers, params=params)

        if response.status_code == 200:
            return response.json().get("tracks", {}).get("items", []), token, token_expiry

        elif response.status_code == 429:  # Rate limited
            retry_after = int(response.headers.get("Retry-After", "5"))
            print(f"Rate limited. Aguardando {retry_after} segundos...")
            time.sleep(retry_after)

        elif response.status_code == 401:  # Token inválido ou expirado
            print("Token inválido/expirado. Renovando...")
            token, token_expiry = get_spotify_access_token(client_id, client_secret)

        else:
            print(f"Erro {response.status_code} para '{track_name}'")
            return [], token, token_expiry

## Autenticando e capturando o token para usar a API

In [0]:
# Autenticação inicial
client_id = "7c4843d7e1a240b49affd223715ebb36"
client_secret = "67dc831362194185be3450976b344a07"
token, token_expiry = get_spotify_access_token(client_id, client_secret)
"""
Explicação:
    - `client_id` e `client_secret`:
        → São as credenciais únicas da aplicação no Spotify for Developers.
        → Usadas para autenticação no fluxo Client Credentials.

    - `get_spotify_access_token(client_id, client_secret)`:
        → Faz uma chamada POST para o endpoint de autenticação do Spotify.
        → Retorna:
            1. `token` (str): O token de acesso (Bearer Token) necessário 
               para autenticar todas as chamadas à API.
            2. `token_expiry` (float): Um timestamp indicando quando esse token expira.

    - Esse passo inicial garante que já temos um token válido antes de
      começar a fazer buscas de músicas (via `get_spotify_track_info`).
"""

## Extraindo os dados da camanda Silver(Com Spark)

In [0]:
# Carrega lista do Spark (Silver)
# Consulta SQL no Spark para buscar músicas distintas
rows = spark.sql("SELECT DISTINCT Track FROM workspace.silver.classic_hit").collect()

# Cria uma lista apenas com os nomes das faixas (coluna 'Track')
tracks_list = [row["Track"] for row in rows]

# Exibe a quantidade total de músicas coletadas
print(f"Total de músicas: {len(tracks_list)}")

"""
Explicação:
    - `spark.sql("SELECT DISTINCT Track FROM workspace.silver.classic_hit")`:
        → Executa uma consulta no Spark SQL.
        → Busca todas as combinações únicas de `Track` na tabela
          `workspace.silver.classic_hit` (camada Silver).
        → O `DISTINCT` garante que não haja duplicados.

    - `.collect()`:
        → Retorna os resultados da consulta como uma lista de objetos Row do Spark.
        → Cada `Row` funciona como um dicionário, onde `row["Track"]` acessa a coluna.

    - `tracks_list = [row["Track"] for row in rows]`:
        → Cria uma lista somente com os títulos das músicas (sem os artistas).
        → Essa lista é usada depois para buscar informações no Spotify.

    - `print(f"Total de músicas: {len(tracks_list)}")`:
        → Exibe no console quantas músicas distintas foram carregadas do dataset.
"""


## Realizando a Busca das musicas no Spotify de forma paralela

In [0]:
# Processa em batches menores
batch_size = 500
all_results = []

# Processa a lista de músicas em lotes (batches)
for i in range(0, len(tracks_list), batch_size):
    batch = tracks_list[i:i + batch_size]
    print(f"Processando batch {i // batch_size + 1} ({len(batch)} músicas)...")

    batch_results = []
    for track in batch:

        # Consulta informações da track na API do Spotify
        results, token, token_expiry = get_spotify_track_info(track, client_id, client_secret, token, token_expiry)
        batch_results.extend(results)

    # Se houve resultados, salva incrementalmente no Delta Table
    # Converte para Spark DataFrame e salva incrementalmente
    if batch_results:
        df = pd.DataFrame([{"json": json.dumps(item)} for item in batch_results])
        spark_df = spark.createDataFrame(df)

        # Persistindo no Delta Lake (camada Bronze)
        spark_df.write.format("delta") \
            .mode("append") \
            .saveAsTable("workspace.bronze.search_track_artist")

    # Armazena todos os resultados também em memória
    all_results.extend(batch_results)

print("Finalizado. Total salvo:", len(all_results))
"""
Explicação:
    - `batch_size = 500`:
        → Define o número de músicas a serem processadas por lote.
        → Ajuda a evitar estouro de memória e problemas com rate limit da API.

    - Loop `for i in range(0, len(tracks_list), batch_size)`:
        → Percorre toda a lista de músicas (`tracks_list`) em blocos de 500 elementos.
        → `batch` contém apenas um pedaço da lista por vez.

    - `get_spotify_track_info(...)`:
        → Faz a chamada à API do Spotify para cada música no lote.
        → Retorna os metadados da faixa (JSON), além do `token` atualizado.

    - `batch_results.extend(results)`:
        → Junta todos os resultados do lote em uma lista temporária.

    - Criação do DataFrame:
        → `pd.DataFrame(...)` transforma a lista de JSONs em um DataFrame do Pandas.
        → `spark.createDataFrame(df)` converte para DataFrame do Spark.

    - Escrita no Delta Lake:
        → `.mode("append")`: adiciona os dados sem sobrescrever os anteriores.
        → Armazena os dados brutos (JSONs) na **camada Bronze** da arquitetura de dados.

    - `all_results.extend(batch_results)`:
        → Também guarda em memória todos os resultados acumulados, 
          caso seja necessário processar ou auditar depois.

    - `print("Finalizado. Total salvo:", len(all_results))`:
        → Exibe no console a quantidade final de músicas realmente gravadas no Delta.
"""